# Problem 1

# Custom Evaluation Metric (Intersection Over Union - IoU)

## Cell 1

### Lines 7-19

    if true == 0: metric += (pred == 0); continue

Handles Empty Masks: If the true mask is empty (true == 0), it scores a 1.0 only if the prediction is also empty (pred == 0). This is crucial for correctly scoring images with no target area.

### Lines 23-24

    intersection = np.sum(t * p) union = true + pred - intersection

Calculates IoU: Standard formula for Intersection and Union between the true and predicted masks.

### Lines 26-28

    iou = intersection / union iou = np.floor(iou * 10) / 10 metric += (iou > 0.5)

Applies Competition Thresholds: Calculates the raw IoU, then floors it to the nearest 0.1. The final score only increments if this thresholded IoU is greater than 0.5 (This is the core metric for the challenge).

## Cell 2

### Lines 5-11

    def competitionMetric(true, pred): t = tf.cast(t, tf.float32) p = tf.cast(p, tf.float32) tf.py_func(get_iou_vector, [t, p], tf.float32)

TensorFlow Wrapper: Creates a TensorFlow-compatible wrapper (tf.py_func) around the detailed NumPy get_iou_vector function, allowing it to be used as a metric during Keras model training.

# Loss Function Definition

## Cell 3

### Lines 5-8

    def lovasz_grad(gt_sorted): g = tf.reduce_sum((1 - gt_sorted) * grad, axis=0) grad = g - tf.cumsum(g)

Gradient Calculation: Defines the specific gradient for the Lovasz loss, which is what allows it to be convex and better approximate the true IoU score compared to standard pixel-wise losses.

### Lines 37-43

    def lovasz_loss(labels, logits, ignore=None): probs = tf.sigmoid(logits) jaccard = (tf.reduce_sum(errors) + smooth) / (tf.reduce_sum(errors) + tf.reduce_sum(labels) - tf.reduce_sum(errors) + smooth)

Lovasz Implementation: The core logic that calculates the loss based on the error rate and then uses the lovasz_grad to penalize misclassifications in a way that optimizes IoU.

### Lines 129-136

    def lovasz_tversky(labels, logits, alpha=0.3, beta=0.7, smooth=1e-5): tversky_loss = 1.0 - tversky(labels, logits, alpha=alpha, beta=beta, smooth=smooth) lovasz_tversky_loss = tversky_loss + lovasz_loss(labels, logits)

Final Loss Combination: Combines the Tversky Loss (a generalization of Dice/F-beta loss) with the Lovasz Loss to create a powerful hybrid loss function for training.

# Model Architecture (Pre-trained U-Net with ResNet34)

## Cell 4

### Line 10

    from tensorflow.keras.optimizers import Adam

Optimizer: Specifies the Adam optimizer, which is the standard choice for most deep learning models.

### Lines 14-16

    def get_unet_resnet(input_size): model = sm.Unet('resnet34', classes=1, activation='sigmoid', input_shape=input_size, encoder_weights='imagenet')

Model Instantiation: This is the most crucial line for the architecture. It creates a U-Net model, using ResNet34 (pre-trained on ImageNet) as the encoder, outputting 1 class (the segmentation mask) with a sigmoid activation.

### Lines 18-20

    model.compile(optimizer=Adam(LR), loss=lovasz_tversky, metrics=[competitionMetric])

Model Compilation: Compiles the model using the defined Adam optimizer, the custom Lovasz Tversky loss, and the custom competitionMetric for monitoring during training.

# Training Callbacks

## Cell 6

### Lines 7-8

    checkpoint = ModelCheckpoint(filepath, monitor='val_competitionMetric', verbose=1, save_best_only=True, mode='max')

Best Model Saving: Creates a checkpoint that only saves the model (save_best_only=True) when the validation competition metric improves, ensuring you keep the highest-performing weights.

### Lines 9-10

    reduce_lr = ReduceLROnPlateau(monitor='val_competitionMetric', factor=0.1, patience=5, verbose=1, mode='max', min_lr=1e-7)

Learning Rate Scheduling: Implements learning rate decay. If the validation competition metric doesn't improve for 5 epochs (patience=5), the learning rate is reduced by a factor of 0.1. This helps escape local minima late in training.

# Training Execution

## Cell 7

### Line 3

    history = model.fit_generator(

Start Training: Initiates the training process using the data generator (which efficiently feeds batches of data to the model).

### Lines 4-8

    train_generator, steps_per_epoch=len(train_generator), epochs=150, validation_data=val_generator, callbacks=callbacks_list)

Training Parameters: Defines the training duration (150 epochs), the validation data source, and applies the important list of callbacks (saving best model, reducing LR).

## Problem 2 and 3

The output for resnet running

<img src="Resnet.png" width="500" height="2000">

### Code changing

On cell 4, I changed the code from

    model = sm.Unet('resnet34', classes=1, activation='sigmoid', input_shape=input_size, encoder_weights='imagenet')

to:

    model = sm.Unet('vgg16', classes=1, activation='sigmoid', input_shape=input_size, encoder_weights='imagenet')

To be able to use VGG

After changing the code to vgg this is the following result

<img src="VGG.png" width="500" height="2000">

The VGG performed better because it had higher validation IoU (0.3926) than the resnet (0.2180) and better validation loss (0.2664) than the resnet (2.7752)